**Упражнение 1.** Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

%time
x = inc(1)
y = inc(2)
z = add(x, y)

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.87 µs


In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

%time
x = inc(1)
y = inc(2)
z = add(x, y)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


с использованием dask.delayed  вызовы происходят быстрее, так как они работают параллельно


**Упражнение 2.** Распараллелить код с помощью dask.delayed. Оцените время работы кода до и после распараллеливания. Объясните результат.

In [ ]:
%time
def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results = []
for x in data:
    if is_even(x):
        y = double(x)
    else:
        y = inc(x)
    results.append(y)

total = sum(results)
print(total)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 9.3 µs
90


In [ ]:
from dask import delayed
from time import sleep

%time
def double(x):
    sleep(1)
    return 2 * x

def is_even(x):
    return not x % 2

data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

results = []
for x in data:
    if is_even(x):
        y = delayed(double)(x)
    else:
        y = delayed(inc)(x)
    results.append(y)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


Как и в прошлом упражнении, даск позваляет работать параллельно

**Упражнение 3.** Использовать dask.bag.

Мы будем использовать пакет Dask для расчета частот последовательностей по пять символов, а затем сортировать последовательности в порядке убывания частоты встреч.

Реализуйте функцию group_characters(line, n=5) для группировки строки по n символов и возврата итератора. Строка — это текстовая строка в файле genome.txt.  (сгенерировать самостоятельно)

In [ ]:
# >>> line = "abcdefghijklmno"
# >>> for seq in group_character(line, 5):
#         print(seq)

# "abcde"
# "efghi"
# "klmno"

Реализовать group_and_split(line)

In [ ]:
# >>> group_and_split('abcdefghijklmno')
# ['abcde', 'fghij', 'klmno']

Использовать dask.bag для вычисления частот встреч групп символов по 5 и отсортировать по ней.

In [ ]:
%%writefile genome.txt
aaaaaaabbbbbbeeubkashdakhghhhhhhh

Writing genome.txt


In [ ]:
from dask import bag as db

def group_and_split(line):
    return [line[i:i+5] for i in range(0, len(line), 5)]

genome_bag = db.read_text('genome.txt')

sequences_bag = genome_bag.map(group_and_split).flatten()

frequencies = sequences_bag.frequencies()

sorted_sequences = frequencies.compute()
sorted_sequences = sorted(sorted_sequences, key=lambda x: x[1], reverse=True)

print(sorted_sequences)


[('aaaaa', 1), ('aabbb', 1), ('bbbee', 1), ('ubkas', 1), ('hdakh', 1), ('ghhhh', 1), ('hhh\n', 1)]


**Упражнение 4.** Использовать dask.array

Вычислите сумму, произведение и среднее значение двумерного квадратного массива (сгенерировать случайно) из 1_000_000 элементов в каждом измерении.

1) с помощью dask (определить самостоятельно разбиение на блоки)

2) без dask

Измерить время

С помощью Dask

In [ ]:
import dask.array as da
import numpy as np
import time

np.random.seed(0)
arr = np.random.rand(1000, 1000)

darr = da.from_array(arr, chunks=(100, 100))

start_time = time.time()
total_sum = darr.sum().compute()
total_prod = darr.prod().compute()
total_mean = darr.mean().compute()
dask_time = time.time() - start_time

print("Сумма:", total_sum)
print("Произведение:", total_prod)
print("Среднее значение:", total_mean)
print("Время выполнения с Dask:", dask_time)



Сумма: 500387.69812077296
Произведение: 0.0
Среднее значение: 0.500387698120773
Время выполнения с Dask: 0.44981932640075684


без dask


In [ ]:
import numpy as np
import time

np.random.seed(0)
arr = np.random.rand(1000, 1000)

start_time = time.time()
total_sum_no_dask = arr.sum()
total_prod_no_dask = arr.prod()
total_mean_no_dask = arr.mean()
no_dask_time = time.time() - start_time

print("Сумма (без Dask):", total_sum_no_dask)
print("Произведение (без Dask):", total_prod_no_dask)
print("Среднее значение (без Dask):", total_mean_no_dask)
print("Время выполнения без Dask:", no_dask_time)


Сумма (без Dask): 500387.69812077284
Произведение (без Dask): 0.0
Среднее значение (без Dask): 0.5003876981207729
Время выполнения без Dask: 0.005124807357788086


**Упражнение 5.**

Постройте массив 20000x20000 нормально распределенных случайных значений (random), разбитых на фрагменты размером 1000x1000
Возьмите среднее по одной оси
Возьмите каждый 100-й элемент

Сделайте задание в двух вариантах:

1) dask

2) numpy

Измерите время и объясните результат.

In [ ]:
import dask.array as da
import numpy as np
import time

# Создание dask.array с нормально распределенными случайными значениями
darr = da.random.normal(size=(20000, 20000), chunks=(1000, 1000))

# Взятие среднего по одной оси
mean_axis = darr.mean(axis=0)

# Взятие каждого 100-го элемента
every_100th = mean_axis[::100]

start_time = time.time()

# Вычисление среднего по одной оси и взятие каждого 100-го элемента
mean_axis_result = mean_axis.compute()
every_100th_result = every_100th.compute()

dask_time = time.time() - start_time

print("Время выполнения с использованием Dask:", dask_time)
print("Среднее по оси:", mean_axis_result)
print("Каждый 100-й элемент:", every_100th_result)


Время выполнения с использованием Dask: 27.777061939239502
Среднее по оси: [0.01032194 0.00192772 0.01033618 ... 0.0011871  0.00853695 0.01125547]
Каждый 100-й элемент: [ 1.03219386e-02  3.61352801e-03  7.85747844e-03 -1.99830483e-03
 -1.22114032e-03 -9.21978817e-03  2.45115504e-03 -1.26228024e-03
  9.82954633e-03  2.54847386e-04  4.50867451e-03  6.37548786e-04
  5.88822032e-03 -3.04483169e-03 -1.85919481e-04 -1.54498411e-02
  2.43991344e-03  4.50937911e-03  1.91257179e-03  6.96725413e-03
  1.15080765e-02 -2.20788594e-03  1.99110805e-02 -1.26084034e-02
 -9.50492976e-03  7.56060277e-03 -6.35389534e-03 -5.95274277e-03
 -4.61721612e-03 -7.58199973e-04  2.24503039e-03 -3.73826373e-03
  8.09223089e-03 -1.17938199e-02 -1.13276117e-02  1.73815555e-03
 -2.55212922e-03 -1.39635320e-02  4.49867406e-03  1.21208053e-04
  2.60335977e-03  1.48846157e-03 -2.00317297e-03  8.69437316e-03
  7.84006330e-03  1.18212703e-02 -7.37592528e-04  1.04126292e-02
  6.90348887e-03 -6.64384550e-03  4.16046303e-04 -1

In [ ]:
import numpy as np
import time

# Создание массива с нормально распределенными случайными значениями
arr = np.random.normal(size=(20000, 20000))

# Взятие среднего по одной оси
mean_axis = np.mean(arr, axis=0)

# Взятие каждого 100-го элемента
every_100th = mean_axis[::100]

start_time = time.time()

# Вычисление среднего по одной оси и взятие каждого 100-го элемента
mean_axis_result = np.mean(arr, axis=0)
every_100th_result = every_100th

numpy_time = time.time() - start_time

print("Время выполнения без использования Dask:", numpy_time)
print("Среднее по оси:", mean_axis_result)
print("Каждый 100-й элемент:", every_100th_result)


Время выполнения без использования Dask: 0.33692097663879395
Среднее по оси: [-0.00769599  0.00090994 -0.00219981 ...  0.00059123 -0.00464151
  0.00252518]
Каждый 100-й элемент: [-7.69598928e-03 -3.56156604e-04 -1.24934045e-03 -3.20410095e-03
  7.30299499e-03 -1.65431827e-03 -1.59724819e-03 -4.44929247e-03
  2.03355002e-03 -3.61419162e-03 -7.35124019e-03 -6.55411189e-03
  1.79452679e-03 -1.47024025e-02 -3.62473646e-03 -3.63581978e-03
  1.50603252e-03 -7.30954884e-03  7.04675309e-03  2.09252880e-03
 -7.40350015e-04  3.12178974e-03 -4.13175831e-03 -5.45404368e-03
  1.83261065e-04  7.69336849e-03  5.22867924e-04 -6.95753721e-03
  2.79781536e-03  4.80354897e-03  3.34342095e-03 -1.08121346e-02
 -6.60956660e-03  6.57305820e-03  4.99931466e-03  6.13911950e-03
  2.74669560e-04 -1.48826694e-03  2.18196714e-03  2.13221715e-02
  2.50825097e-02  7.04435762e-03  1.05399244e-02 -3.21513260e-05
  1.16250293e-02 -4.82883072e-03  8.08919835e-03  3.46156161e-03
  9.56914764e-03 -6.10575322e-03 -4.776573